https://www.youtube.com/watch?v=OYhFoMySoVs

In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os # for creating directories

2022-11-01 13:38:57.855053: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
env = gym.make('CartPole-v0') # initialise environment


In [20]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [21]:
state_size = env.observation_space.shape[0]
state_size

4

In [22]:
action_size = env.action_space.n
action_size

2

In [5]:
batch_size = 32

In [6]:
n_episodes = 1001 # n games we want agent to play (default 1001)


In [7]:
output_dir = 'model_output/cartpole/'


In [8]:

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [9]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
    
    def _build_model(self):
        # neural net to approximate Q-value function:
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(24, activation='relu')) # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear')) # 2 actions, so 2 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                target = (reward + self.gamma * # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [10]:
agent = DQNAgent(state_size, action_size) # initialise agent

2022-11-01 13:39:08.254317: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/andreyremorov/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [11]:
done = False
for e in range(n_episodes): # iterate over new episodes of the game
    state = env.reset() # reset state at start of each new episode of the game
    state = np.reshape(state, [1, state_size])
    
    for time in range(5000):  # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps
#         env.render()
        action = agent.act(state) # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position        
        reward = reward if not done else -10 # reward +1 for each additional frame with pole upright        
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
        state = next_state # set "current state" for upcoming iteration to the current next state        
        if done: # episode ends if agent drops pole or we reach timestep 5000
            print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score and agent's epsilon
                  .format(e, n_episodes, time, agent.epsilon))
            break # exit loop
    if len(agent.memory) > batch_size:
        agent.replay(batch_size) # train the agent by replaying the experiences of the episode
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")         

episode: 0/1001, score: 99, e: 1.0
1/1 [==============================] - 0s 10ms/step
episode: 1/1001, score: 13, e: 0.99
1/1 [==============================] - 0s 11ms/step
episode: 2/1001, score: 15, e: 0.99
1/1 [==============================] - 0s 10ms/step
episode: 3/1001, score: 24, e: 0.99
1/1 [==============================] - 0s 14ms/step
episode: 4/1001, score: 13, e: 0.98
1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 10ms/step
episode: 5/1001, score: 21, e: 0.98
1/1 [==============================] - 0s 15ms/step
episode: 6/1001, score: 13, e: 0.97
1/1 [==============================] - 0s 13ms/step
episode: 7/1001, score: 23, e: 0.97
1/1 [==============================] - 0s 14ms/step
episode: 8/1001, score: 34, e: 0.96
1/1 [==============================] - 0s 17ms/step
episode: 9/1001, score: 34, e: 0.96
1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 16ms/step
episode: 10/1001, score: 12, e: 0.95
1/1 [==============================] - 0s 16ms/step
episode: 11/1001, score: 21, e: 0.95
1/1 [==============================] - 0s 17ms/step
episode: 12/1001, score: 55, e: 0.94
1/1 [==============================] - 0s 15ms/step
episode: 13/1001, score: 16, e: 0.94
1/1 [==============================] - 0s 17ms/step
episode: 14/1001, score: 27, e: 0.93
1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 15ms/step
episode: 15/1001, score: 11, e: 0.93
1/1 [==============================] - 0s 14ms/step
episode: 16/1001, score: 22, e: 0.92
1/1 [==============================] - 0s 18ms/step
episode: 17/1001, score: 40, e: 0.92
1/1 [==============================] - 0s 15ms/step
episode: 18/1001, score: 25, e: 0.91
1/1 [==============================] - 0s 16ms/step
episode: 19/1001, score: 10, e: 0.91
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 15ms/step
episode: 20/1001, score: 17, e: 0.9
1/1 [==============================] - 0s 15ms/step
episode: 21/1001, score: 37, e: 0.9
1/1 [==============================] - 0s 15ms/step
episode: 22/1001, score: 7, e: 0.9
1/1 [==============================] - 0s 16ms/step
episode: 23/1001, score: 11, e: 0.89
1/1 [==============================] - 0s 16ms/step
episode: 24/1001, score: 17, e: 0.89
1/1 [==============================] - 0s 16ms/step


...

1/1 [==============================] - 0s 11ms/step
episode: 526/1001, score: 51, e: 0.072
1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step
episode: 527/1001, score: 117, e: 0.071
1/1 [==============================] - 0s 13ms/step
episode: 528/1001, score: 30, e: 0.071
1/1 [==============================] - 0s 11ms/step
episode: 529/1001, score: 41, e: 0.071
1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 10ms/step
episode: 530/1001, score: 59, e: 0.07
1/1 [==============================] - 0s 11ms/step
episode: 531/1001, score: 66, e: 0.07
1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 12ms/step
episode: 532/1001, score: 43, e: 0.069
1/1 [==============================] - 0s 11ms/step


KeyboardInterrupt: 